In [1]:

import numpy as np
import pandas as pd
import os 
os.chdir('C:/Users/rodri/Dropbox/PhD thesis/python')
from data_functions_albert import remove_outliers, gini
os.chdir('C:/Users/rodri/Dropbox/Chiefs_Field_Aug_18/Data/consumption/')
percentiles = [0.05, 0.1, .25, .5, .75, 0.8, 0.9, 0.95, 0.99]
euro_MWK = 806.64


###  I NEEED TO RECHECK WITH USING SAME LOOPING AS IN THE 2019 WAVE. I THINK VALUES HERE IS WRONG

#Import data
data = pd.read_csv("data_SIEG_clean.csv")

x0 = data.columns.get_loc("consume_1")
x1 = data.columns.get_loc("salt_unit9")
datacon = data

list_items = ['maizemgaiwa', 'maizerefined', 'maizemadeya', 'maizegrain', 'greenmaize', 'rice', 'cassavatubers', 'wsweetpotatoes', 'osweetpotatoes'
, 'ipotatoes', 'potatocrisps', 'bbean', 'pigeonpea', 'groundnut', 'groundnutf', 'onion', 'cabbage', 'tanaposi', 'leafyvegetables', 'tomato', 'eggs', 
'driedfish', 'fleshfish', 'goat', 'chicken', 'otherpoultry', 'smockedfish', 'mango', 'banana', 'guava', 
'wildfruits',  'sugar', 'sugarcane', 'cookingoil', 'softdrinks',
'thobwa', 'locallybrewed', 'salt']


#Obtain the names of the variables per each question of item. 
#Question c is monetary question so not conversion to kgs needed
a_var = []
b_var = []
d_var = []
e_var = []
f_var = []
g_var = []
h_var = []
i_var = []
j_var = []

#Generate variable lables in a list
for item in list_items :
    a = item+'_a'
    b= item+'_b'
    d = item+'_d'
    e = item+'_e'
    f = item+'_f'
    g = item+'_g'
    h = item+'_h'
    i = item+'_i'
    j = item+'_j'
    a_var.append(a)
    b_var.append(b)
    d_var.append(d)
    e_var.append(e)
    f_var.append(f)
    g_var.append(g)
    h_var.append(h)
    i_var.append(i)
    j_var.append(j) 


list_questions = ['a','b','d','e','f','g','h','i','j']

# Generate kg variables empty
for item in list_items:
    for q in list_questions:
        datacon[item+'_'+q+'kg']= np.nan
    
# IN UNITS QUESTIONS: Drop nan observations. Also drop unit 25 (number not in our choices). Also drop 24(whole chicken).
for var in list_items:
    for i in range(1,9):
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(np.nan, 99)
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(25, 99)
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(24, 99)

## For salt we don't have question of units for given-out ganyu:
for var in list_items[0:-1]:
    datacon[[var+'_unit9']] = datacon[[var+'_unit9']].replace(np.nan, 99)
    datacon[[var+'_unit9']] = datacon[[var+'_unit9']].replace(25, 99)
    datacon[[var+'_unit9']] = datacon[[var+'_unit9']].replace(24, 99)  


datacon[['salt_unit9']] =99   

'''
##### THIS CODE IS TO GENERATE THE MATRIX OF ITEMS/UNITS COMBINATIONS TO CONVERT TO KGS. NO NEED TO RE-RUN IT.
data_conversion = pd.read_excel('conversionkgrates_consumption_malawi_short.xls')

#del data, datasets, data0, data1, data2, data3, data4, x0, x1

items_conv = data_conversion.item_label.unique()


list_items_conv = ['Maize Mgaiwa', 'Maize refined', 'Maize Madeya', 'Maize grain',
       'Greenmaize', 'Rice', 'Cassava tubers',
       'White sweet potatoes', 'Orange sweet potatoes', 'Irish Potatoe',
       'Potatoe Chips', 'Bean, brown', 'Pigeon peas (ndolo)', 'Groundnut',
       'Groundnut flour', 'Onion', 'Cabbage', 'Tanaposi/Rape',
       'Other cultivated green leafy vegetables', 'Tomatoe', 'Eggs',
       'Dried Fish', 'Fresh Fish', 'Goat', 'Chicken',
       'Other poultry-guinea fowl, doves, etc', 'Smoked Fish', 'Mango',
       'Banana', 'Guava', 'Wild fruit (Masau, Malambe, etc)', 'Sugar',
       'Sugar Cane', 'Cooking oil',
       'Soft drinks(coca-cola, fanta, sprite etc)', 'Thobwa',
       'Locally brewed liquor(kachasu)', 'Salt']


#Obtain the names of the variables per each question of item. Question c is monetary question so not conversion to kgs needed
a_var = []
b_var = []
d_var = []
e_var = []
f_var = []
g_var = []
h_var = []
i_var = []
j_var = []
#Generate variable lables in a list
for item in list_items :
    a = item+'_a'
    b= item+'_b'
    d = item+'_d'
    e = item+'_e'
    f = item+'_f'
    g = item+'_g'
    h = item+'_h'
    i = item+'_i'
    j = item+'_j'
    a_var.append(a)
    b_var.append(b)
    d_var.append(d)
    e_var.append(e)
    f_var.append(f)
    g_var.append(g)
    h_var.append(h)
    i_var.append(i)
    j_var.append(j) 


list_questions = ['a','b','d','e','f','g','h','i','j']

# Generate kg variables empty
for item in list_items:
    for q in list_questions:
        datacon[item+'_'+q+'kg']= np.nan
    
# Drop nan observations. Also drop unit 25 (number not in our choices). Also drop 24(whole chicken).
for var in list_items:
    for i in range(1,9):
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(np.nan, 99)
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(25, 99)
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(24, 99)
        
#       
# Import conversion files
conversionkg = data_conversion[['unit','item_label','conversion_kgs_country']]


# Reshape as: rows:units, columns:crops

conversionkg = conversionkg.replace(list_items_conv, list_items)

conversionkg_pivot = conversionkg.pivot_table(values='conversion_kgs_country',
                                index='unit',
                                columns='item_label')


conversionkg_pivot.loc[99,:] = np.nan

conversionkg_pivot.to_csv('conversionkg_isaprices_matrix.csv')
'''

### CONVERSION TO KGS: ISA-LSMS SOUTH MALAWI PRICE CONVERSION WITH ALBERT REPLACING VALUES THAT DIDN'T HAVE COMMON SENSE
conversionkg_pivot = pd.read_csv('conversionkg_isaprices_matrix.csv',  index_col=0)

#%% CONVERT TO KGS FOR ALL QUESTIONS FOR ALL ITEMS

print('a: Total Consumption')
for var in a_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit1')]), item]
    print(datacon[[var+'kg']].describe())

C:\Users\rodri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (62,78,86,122,124,133,160,191,197,222,253,259,315,377,408,412,424,433,439,443,459,569,600,625,657,796,797,810,815,827,830,832,850,852,869,871,892,902,906,929,933,947,951,954,955,958,962,965,969,978,982,985,986,989,992,996,1003,1006,1023,1040,1187,1219,1285,1286,1295,1296,1305,1306,1324,1328,1345,1349,1386,1389,1390,1425,1435,1444,1460,1462,1491,1498,1508,1517,1526,1533,1535,1588,2384,2387,2524,2527,2531,2823,2826,3012,3021,3022,3027,3034,3083,3092,3093,3105,3154,3164,3228,3234,3235,3237,3240,3242,3249,3258,3259,3264,3319,3328,3386,3389,3390,3396,3398,3399,3462,3465,3468,3470,3502,3508,3510,3532,3533,3534,3535,3561,3562,3572,3575,3576,3577,3591,3594,3595,3610,3613,3614,3629,3632,3648,3652,3663,3673,3674,3683,3684,3694,3695,3710,3715,3720,3725,3730,3735,3740,3741,3751,3752,3762,3766,3767,3780,3784,3785,3798,3802,3808) have mixed types. Specify dtype option on import or set low_memory=F

a: Total Consumption
       maizemgaiwa_akg
count       193.000000
mean         10.357032
std           7.468667
min           1.000000
25%           5.000000
50%          10.000000
75%          15.000000
max          40.000000
       maizerefined_akg
count        132.000000
mean           7.124456
std            5.576689
min            0.600000
25%            3.125000
50%            6.250000
75%            8.333333
max           30.000000
       maizemadeya_akg
count        58.000000
mean          2.431609
std           2.052436
min           0.500000
25%           1.000000
50%           2.000000
75%           4.000000
max          10.000000
       maizegrain_akg
count       26.000000
mean         0.760531
std          0.977805
min          0.250000
25%          0.309524
50%          0.500000
75%          0.589286
max          5.000000
       greenmaize_akg
count       12.000000
mean         0.282051
std          0.215917
min          0.000000
25%          0.153846
50%          0.2307

In [2]:

print('b: Bought')
for var in b_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit2')]), item]

print('d: Own-produced')
for var in d_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit3')]), item]

print('e: Gift-in')
for var in e_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit4')]), item]

print('f: Gift-in for free')
for var in f_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit5')]), item]

print('g: Gift-in for ganyu')
for var in g_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit6')]), item]

print('h: Gift-out')
for var in h_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit7')]), item]

print('i: Food giften out for free')
for var in i_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit8')]), item]

print('j: Food giften out for ganyu')
for var in j_var:
    item = var[:-2]
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit9')]), item]


b: Bought
d: Own-produced
e: Gift-in
f: Gift-in for free
g: Gift-in for ganyu
h: Gift-out
i: Food giften out for free
j: Food giften out for ganyu


In [3]:
#%% Checks

# Sum the subset questions


#check total consumption
datacon['total_foodkg'] = 0
datacon['purchased_kg'] = 0
datacon['ownproduced_kg'] = 0
datacon['giftin_kg'] = 0
datacon['giftin_free_kg'] = 0
datacon['giftin_ganyu_kg'] = 0
datacon['giftout_kg'] = 0
datacon['giftout_free_kg'] = 0
datacon['giftout_ganyu_kg'] = 0


for item in list_items:
    datacon['total_foodkg'] += datacon[item+'_akg'].replace(np.nan, 0)
    datacon['purchased_kg'] += datacon[item+'_bkg'].replace(np.nan, 0)
    datacon['ownproduced_kg'] += datacon[item+'_dkg'].replace(np.nan, 0)
    datacon['giftin_kg'] += datacon[item+'_ekg'].replace(np.nan, 0)
    datacon['giftin_free_kg'] += datacon[item+'_fkg'].replace(np.nan, 0)
    datacon['giftin_ganyu_kg'] += datacon[item+'_gkg'].replace(np.nan, 0)
    datacon['giftout_kg'] += datacon[item+'_hkg'].replace(np.nan, 0)
    datacon['giftout_free_kg'] += datacon[item+'_ikg'].replace(np.nan, 0)
    datacon['giftout_ganyu_kg'] += datacon[item+'_jkg'].replace(np.nan, 0)
    
sumtotalfoodkg = datacon[['total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']].describe(percentiles)   


print('==== Summary Food Consumption last 7 days in kgs aggregated across items =======')
print('')
print(sumtotalfoodkg)
#%%  CONVERT TO MONETARY VALUE   ==========================================

# Get prices -------------------------
for item in list_items:
        datacon[item+'_price']= np.nan

## Compute price paid per each household each item
for item in list_items:
    datacon[item+'_price'] = datacon[item+'_c'] / datacon[item+'_bkg'].replace(0,np.nan)

price_data = pd.DataFrame(list_items, columns=['good'])
price_data['p_c'] = np.nan

## Get median prices per item
for item in list_items:
    print('Median Price 1 kg of '+item)
    datacon['med_price_'+item] = datacon[item+'_price'].median()
    print(datacon['med_price_'+item].mean())
    price_data.loc[price_data['good']==item,'p_c'] = data['med_price_'+item].mean()

price_data.to_csv('village_c_prices_18.csv', index=False)


==== Summary Food Consumption last 7 days in kgs aggregated across items =======

       total_foodkg  purchased_kg  ownproduced_kg   giftin_kg  giftin_free_kg  \
count    269.000000    269.000000      269.000000  269.000000      269.000000   
mean      96.542810     83.242842       11.512464    9.996090        6.078535   
std      196.287307    221.406171       20.343986   45.960345       34.622125   
min        2.405555      0.000000        0.000000    0.000000        0.000000   
5%        11.627678      2.003329        0.000000    0.000000        0.000000   
10%       17.324689      5.296252        0.000000    0.000000        0.000000   
25%       30.128179     17.177300        0.883123    0.000000        0.000000   
50%       51.145066     38.477779        4.525333    1.118933        0.000000   
75%       86.813014     65.577628       14.305556    4.566933        1.422220   
80%       97.270996     79.752073       19.227288    6.359455        2.080000   
90%      155.142978    119.

In [4]:

## compute consumption in MWK -----------------   
for item in list_items:
    for q in list_questions:
        datacon[item+'_'+q+'MWK']= np.nan

print('a: Total Consumption')
for item in list_items:
    datacon[item+'_aMWK'] = datacon[item+'_akg']*datacon['med_price_'+item]
    print('Food Consumption in MWK during last 7 days item: '+item)
    print(datacon[item+'_aMWK'].describe(percentiles=percentiles))
            
print('b: Bought')
for item in list_items:
    datacon[item+'_bMWK'] = datacon[item+'_bkg']*datacon['med_price_'+item]


print('d: Own-produced')
for item in list_items:
    datacon[item+'_dMWK'] = datacon[item+'_dkg']*datacon['med_price_'+item]
    

print('e: Gift-in')
for item in list_items:
    datacon[item+'_dMWK'] = datacon[item+'_dkg']*datacon['med_price_'+item]
    
print('f: Gift-in for free')
for item in list_items:
    datacon[item+'_fMWK'] = datacon[item+'_fkg']*datacon['med_price_'+item]

print('g: Gift-in for ganyu')
for item in list_items:
    datacon[item+'_gMWK'] = datacon[item+'_gkg']*datacon['med_price_'+item]
    

print('h: Gift-out')
for item in list_items:
    datacon[item+'_hMWK'] = datacon[item+'_hkg']*datacon['med_price_'+item]
    
    
print('i: Food giften out for free')
for item in list_items:
    datacon[item+'_iMWK'] = datacon[item+'_ikg']*datacon['med_price_'+item]
 
    
print('j: Food giften out for ganyu')
for item in list_items:
    datacon[item+'_jMWK'] = datacon[item+'_jkg']*datacon['med_price_'+item]    
    
#check total consumption
datacon['c_food'] = 0
datacon['c_food_purch'] = 0
datacon['c_food_ownprod'] = 0
datacon['c_food_giftin'] = 0
datacon['c_food_giftin_free'] = 0
datacon['c_food_giftin_ganyu'] = 0
datacon['c_food_giftout'] = 0
datacon['c_food_giftout_free'] = 0
datacon['c_food_giftout_ganyu'] = 0

for item in list_items:
    datacon['c_food'] += datacon[item+'_aMWK'].replace(np.nan, 0)
    datacon['c_food_purch'] += datacon[item+'_bMWK'].replace(np.nan, 0)
    datacon['c_food_ownprod'] += datacon[item+'_dMWK'].replace(np.nan, 0)
    datacon['c_food_giftin'] += datacon[item+'_eMWK'].replace(np.nan, 0)
    datacon['c_food_giftin_free'] += datacon[item+'_fMWK'].replace(np.nan, 0)
    datacon['c_food_giftin_ganyu'] += datacon[item+'_gMWK'].replace(np.nan, 0)
    datacon['c_food_giftout'] += datacon[item+'_hMWK'].replace(np.nan, 0)
    datacon['c_food_giftout_free'] += datacon[item+'_iMWK'].replace(np.nan, 0)
    datacon['c_food_giftout_ganyu'] += datacon[item+'_jMWK'].replace(np.nan, 0)

datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu']] = datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu']].replace(0.0,np.nan)

 
sumcfood= ((datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu']]*4/euro_MWK).replace(0.0,np.nan)).describe(percentiles)  
 
print('==== Summary Food Consumption at Month level in Euros =======')
print(sumcfood)


a: Total Consumption
Food Consumption in MWK during last 7 days item: maizemgaiwa
count     193.000000
mean     1760.695361
std      1269.673436
min       170.000000
5%        345.142869
10%       631.428554
25%       850.000000
50%      1700.000000
75%      2550.000000
80%      2644.444408
90%      3400.000000
95%      3740.000000
99%      5236.000000
max      6800.000000
Name: maizemgaiwa_aMWK, dtype: float64
Food Consumption in MWK during last 7 days item: maizerefined
count     132.000000
mean     1367.895591
std      1070.724301
min       115.200009
5%        156.872441
10%       289.200011
25%       600.000023
50%      1200.000046
75%      1600.000000
80%      1920.000073
90%      2400.000092
95%      3488.000066
99%      4800.000183
max      5760.000220
Name: maizerefined_aMWK, dtype: float64
Food Consumption in MWK during last 7 days item: maizemadeya
count      58.000000
mean      486.321840
std       410.487129
min       100.000000
5%        133.333337
10%       133.333337
25

In [5]:
#%% non-food consumption


data['c_housing'] = data['bills']
data['c_clothes'] = data['clothesothers']
data['c_education'] = data['educationothers'] 
data['c_health'] = data['healthothers']
data['c_funeralout'] = data['funerals1']
data['c_funeralin'] = data['funerals2']
data['c_weddingout'] = data['wedding1']
data['c_weddingin'] = data['wedding2'] 

data['c_nonfood'] = data[['c_housing', 'c_clothes', 'c_education', 'c_health']].sum(axis = 1, skipna = True)

sum_cnonfood = ((data[['c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']]/euro_MWK).replace(0.0,np.nan)).describe()
print('======== SUMMARY  NON-FOOD CONSUMPTION (MONTH LEVEL)')
print('summary in EUR')
sum_cnonfood


======== SUMMARY  NON-FOOD CONSUMPTION (MONTH LEVEL)
summary in EUR


,c_nonfood,c_housing,c_clothes,c_education,c_health,c_funeralout,c_funeralin,c_weddingout,c_weddingin
count,262.000000,261.000000,178.000000,125.000000,144.000000,79.000000,14.000000,43.000000,10.000000
mean,29.468460,8.565259,20.429801,7.882674,5.995592,2.239953,17.355946,2.466447,3.582763
std,35.413889,13.328672,22.187070,11.544735,7.432138,6.811640,25.787168,2.659762,3.834183
min,0.247942,0.123971,0.247942,0.148765,0.123971,0.012397,0.123971,0.247942,0.247942
25%,7.066349,2.479421,5.826639,1.859566,1.239710,0.247942,0.619855,1.146732,0.495884
50%,15.186452,4.338986,12.397104,3.719131,3.719131,0.619855,3.099276,1.239710,2.355450
75%,40.135624,9.917683,24.794208,6.198552,6.198552,1.239710,23.244570,3.099276,5.423733
max,278.934841,123.971040,123.971040,68.184072,37.191312,55.786968,86.779728,12.397104,12.397104


In [6]:
#%%  Export dataset

## Convert to rainy season (7 months)


datacon_short = datacon[[ 'hhid','c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu','c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']] 


## Food at monthly level
datacon_short[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu']] = datacon_short[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu']]*4

datacon_short['ctotal'] = datacon_short[['c_nonfood', 'c_food']].sum(axis = 1, skipna = True)

## Consumption at rainy season
datacon_short[['ctotal','c_food','c_food_purch','c_food_ownprod',  'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health']] = datacon_short[['ctotal','c_food','c_food_purch','c_food_ownprod',  'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health']]*7




datacon_short.to_csv('cons_short_18.csv', index=False)

## Not at rainy season level
datacon.to_csv('cons_long_18.csv')

C:\Users\rodri\Anaconda3\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\rodri\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
